## Defining a new problem

To define a new problem, inherit from the `OnLineProblem` abstract class defined in the `xcs.problems` submodule. Suppose, as an example, that we wish to test the algorithm's ability to find a single important input bit from among a large number of irrelevant input bits.

In [1]:
from xcs.problems import OnLineProblem

class HaystackProblem(OnLineProblem):
    pass

We defined a new class, `HaystackProblem`, to represent this test case, which inherits from `xcs.problems.OnLineProblem` to ensure that we cannot instantiate the problem until the appropriate methods have been implemented.

Now let's define an `__init__` method for this class. We'll need a parameter, `training_cycles`, to determine how many reward cycles the algorithm has to identify the "needle", and another parameter, `input_size`, to determine how big the "haystack" is.

In [2]:
from xcs.problems import OnLineProblem

class HaystackProblem(OnLineProblem):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles


The `input_size` is saved as a member for later use. Likewise, the value of `training_cycles` was saved in two places: the `remaining_cycles` member, which tells the instance how many training cycles remain for the current run, and the `intitial_training_cycles` member, which the instance will use to reset `remaining_cycles` to the original value for a new run.

We also defined the `possible_actions` member, which we set to `(True, False)`. This is the value we will return when the algorithm asks for the possible actions. We will expect the algorithm to return `True` when the needle's bit is set, and `False` when the needle's bit is clear, in order to indicate that it has correctly identified the needle's location.

Now let's define some methods for the class. The `OnLineProblem` defines several abstract methods:
* `get_possible_actions()` should return the actions the algorithm can take.
* `reset()` should restart the problem for a new run.
* `sense()` should return a new input (the "situation").
* `execute(action)` should accept an action from among those returned by `get_possible_actions()`, in response to the last situation that was returned by `sense()`. It should then return a reward value indicating how well the algorithm is doing at solving the problem.
* `more()` should return a Boolean value to indicate whether the algorithm has remaining reward cycles in which to solve the problem.

Each of these abstract methods must be defined, or we will get a TypeError when we attempt to instantiate the class:

In [3]:
problem = HaystackProblem()

TypeError: Can't instantiate abstract class HaystackProblem with abstract methods execute, get_possible_actions, more, sense

The implementations for the methods other than `sense()` and `execute()` will be trivial, so let's start with those:

In [4]:
from xcs.problems import OnLineProblem

class HaystackProblem(OnLineProblem):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles

    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        
    def more(self):
        return self.remaining_cycles > 0

Now we are going to get into the meat of the problem. We want to give the algorithm a random string of bits of size `input_size` and have it pick out the location of the needle through trial and error, by telling us what it thinks the value of the bit associated with the needle is. For this to be a useful test, the needle needs to be in a fixed location, which we have not yet defined. Let's choose a random bit from among inputs on each run.

In [5]:
import random

from xcs.problems import OnLineProblem

class HaystackProblem(OnLineProblem):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle = random.randrange(input_size)

    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0

The `sense()` method is going to create a string of random bits of size `input_size` and return it. But first it will pick out the value of the bit associated with the `needle` and store it in a new member, `needle_value`, so that `execute(action)` will know what the correct value for `action` is.

In [6]:
import random

from xcs.problems import OnLineProblem
from xcs.bitstrings import BitString

class HaystackProblem(OnLineProblem):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle = random.randrange(input_size)
        self.needle_value = None

    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle]
        return haystack

Now we need to define the `execute(action)` method. In order to give the algorithm appropriate feedback to make problem solvable, we should return a high reward when it guesses the correct value for the needle bit, and a low value otherwise. Thus we will return a `1` when the action is the correct one, and a `0` otherwise.

In [7]:
import random

from xcs.problems import OnLineProblem
from xcs.bitstrings import BitString

class HaystackProblem(OnLineProblem):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle = random.randrange(input_size)
        self.needle_value = None

    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle]
        return haystack
    
    def execute(self, action):
        return action == self.needle_value

We have now defined all of the methods that `OnLineProblem` requires. Let's give it a test run.

In [ ]:
import logging
import xcs

# Setup logging so we can see the test run as it progresses.
logging.root.setLevel(logging.INFO)

problem = HaystackProblem()
xcs.test(problem=problem)